### The purpose of this file is to remove identifying information and replace it with Generated Data. Unique data should remain unique, but be anonymized and cleaned to just the data that is necessary for this project.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
%ls data

### Load and combine the data files of the same structure.

I've commented all this code, but left it intact to show what was done to process the files in order to put them in better working condition.

In [ ]:
# transactions_2019 = pd.read_csv('data/20190101-20191231-transactions.csv')
# transactions_2020 = pd.read_csv('data/20200101-20201205-transactions.csv')

# transactions_items_2019 = pd.read_csv('data/20190101-20191231-transaction-items.csv')
# transactions_items_2020 = pd.read_csv('data/20200101-20201205-transaction-items.csv')

# transactions_tenders_2019 = pd.read_csv('data/20190101-20191231-transaction-tenders.csv')
# transactions_tenders_2020 = pd.read_csv('data/20200101-20201205-transaction-tenders.csv')

In [ ]:
### join the combined files on name
# print(transactions_2019.shape)

In [ ]:
# print(transactions_2020.shape)

In [ ]:
# transactions_full = transactions_2019.append(transactions_2020)
# transactions_items_full = transactions_items_2019.append(transactions_items_2020)
# transactions_tenders_full = transactions_tenders_2019.append(transactions_tenders_2020)

In [ ]:
# print(f"transactions_full.shape {transactions_full.shape}")
# print(f"transactions_items_full {transactions_items_full.shape}")
# print(f"transactions_tenders_full {transactions_tenders_full.shape}")

In [ ]:
# transactions_full.to_csv("data/transactions_full.csv")
# transactions_items_full.to_csv("data/transactions_items_full.csv")
# transactions_tenders_full.to_csv("data/transactions_tenders_full.csv")

In [ ]:
# In transactions, the fields we need to worry about are:
#     Cashier Name, Customer Name, Customer Email, Cardholder Name
transactions_full.head(100)

In [ ]:
## Create some lists of unique data that we want to replace

# Some of these became unnecessary as I realized I didn't need those columns.
# cashier_names = transactions_full["Cashier Name"].drop_duplicates()
# customer_names = transactions_full["Customer Name"].drop_duplicates()
# customer_emails = transactions_full["Customer Email"].drop_duplicates()

cardholder_names = transactions_full["Cardholder Name"].drop_duplicates()
print(cardholder_names)


In [ ]:
## Here we are establishing that whenever there is NOT a Cardholder name, 
# there is not a customer name. In other words, customer name is a subset of carholder name
test = transactions_full[transactions_full["Cardholder Name"] == ""]
customer_names = test["Customer Name"].drop_duplicates()
print(customer_names)

In [ ]:
# transactions_full = transactions_full.drop("Cashier Name", axis = 1)

In [ ]:
# transactions_full = transactions_full.drop("Customer Name", axis = 1)
# transactions_full = transactions_full.drop("Customer Email", axis = 1)
# transactions_full = transactions_full.drop("Last 4 Digits", axis = 1)

In [ ]:
transactions_full.head(4)

### Cleaning transaction_items

It looks like here we just needed to drop a few uneccessary columns

In [ ]:
# transactions_items_full = transactions_items_full.drop("Cashier Name", axis = 1)
# transactions_items_full = transactions_items_full.drop("Category", axis = 1)

transactions_items_full.head(3)


### Transaction Tenders

Again, we don't need the Cashier Name, Customer Email, Last 4 Digits

In [ ]:
# transactions_tenders_full = transactions_tenders_full.drop("Cashier Name", axis = 1)
# transactions_tenders_full = transactions_tenders_full.drop("Customer Email", axis = 1)
# transactions_tenders_full = transactions_tenders_full.drop("Last 4 Digits", axis = 1)

transactions_tenders_full.head(3)

# Sanitizing PI

At this point we have a list of customers in cardholder_names and we need to generate
a key of fake names and substitute them into the dataframes before rewriting the csv

In [ ]:
import names
from faker import Faker

fake = Faker()

print(len(cardholder_names))

In [ ]:
names = pd.DataFrame([], columns=['fakes'])

while len(names.drop_duplicates()) < len(cardholder_names):
    names = names.append({'fakes': fake.name()}, ignore_index=True)

names = names.drop_duplicates()


In [ ]:
names.shape

In [ ]:
cardholder_names.shape

In [ ]:
frame = { 'customers': cardholder_names } 
  
result = pd.DataFrame(frame) 

result.shape


In [ ]:
joined = pd.concat([names.reset_index(), cardholder_names.reset_index()], axis = 1)
joined.head()

In [ ]:
joined.shape

In [ ]:
customer_key = joined.drop('index', axis = 1)
# customer_key = customer_key.drop('index', axis = 1)

In [ ]:
customer_key.head(10)

In [ ]:
pd = transactions_full.join(customer_key.set_index('Cardholder Name'), on= 'Cardholder Name')

In [ ]:
# pd[pd['fakes'] == 'Denise Floyd'].head(20)

In [ ]:
pd.head(1)

In [ ]:
transactions_full_clean = pd.drop('Cardholder Name', axis = 1)

In [ ]:
transactions_full_clean['Cardholder Name'] = transactions_full_clean['fakes']

In [ ]:
transactions_full_clean = transactions_full_clean.drop('fakes', axis = 1)

In [ ]:
transactions_full_clean.to_csv("data/transactions_full_clean.csv")

In [ ]:
transactions_full_clean.head(3)

## Now we need to do the same thing with Transactions Tenders

In [ ]:
pd = transactions_tenders_full.join(customer_key.set_index('Cardholder Name'), on= 'Cardholder Name')

In [ ]:
transactions_tenders_full_clean = pd.drop('Cardholder Name', axis = 1)
transactions_tenders_full_clean['Cardholder Name'] = transactions_tenders_full_clean['fakes']
transactions_tenders_full_clean = transactions_tenders_full_clean.drop('fakes', axis = 1)
transactions_tenders_full_clean.head(3)

In [ ]:
transactions_tenders_full_clean.to_csv("data/transactions_tenders_full_clean.csv")

In [ ]:
customer_key.to_csv('data/customer_key.csv')

## Finally, Transaction Items

In [ ]:
transactions_items_full.head(3)


In [ ]:
transactions_items_full.to_csv("data/transactions_items_full_clean.csv")